In [10]:
import math
from estimator import *
from sage.misc.table import table

In [29]:
jobs = 16

e = 3.20 * (2**39) # For LWE
e_lwe_distribution = ND.DiscreteGaussian(e)

# LWE
n = 1024
# q = 1024
q = 2**64

lwe_key_distributionn = ND.SparseBinary(n // 2, n)

# RLWE
N = 2 * n
Q = 4611686018427322369
logQ = Q.bit_length()

# r_distribution = ND.Uniform(0, Q - 1)
rlwe_key_distributionn = ND.SparseBinary(N // 2, N)
e_rlwe_distribution = ND.DiscreteGaussian(3.20*(1<<10))
# r_distribution = ND.UniformMod(Q)

# key switching
e_key_switching = 3.20 * (1 << 37)
key_switching_key_distribution = ND.DiscreteGaussian(e_key_switching)

header = ["logQ"]
data = [logQ]
print(table(columns = [data], header_column = header, frame = True))

┌──────╥────┐
│ logQ ║ 62 |
└──────╨────┘


In [8]:
lwe_param = LWE.Parameters(n=n, q=q, Xs=lwe_key_distributionn, Xe=e_lwe_distribution)
print(lwe_param)
try:
    lwe_result = LWE.estimate(
        lwe_param,
        deny_list=(
            # "arora-gb",
            # "bkw",
        ),
        jobs=jobs,
    )
except Exception as err:
    print("err=", err)
    print("Error Occur!")
else:
    lwe_security = min([math.log(res.get("rop", 0), 2) for res in lwe_result.values()])
    header = ["n","q","secret key distribution","noise distribution", "lwe security"]
    data = [n,q,lwe_key_distributionn,e_lwe_distribution,lwe_security]
    print(table(columns = [data], header_column = header, frame = True))

LWEParameters(n=1024, q=18446744073709551616, Xs=D(σ=0.50, μ=0.50), Xe=D(σ=1759218604441.60), m=+Infinity, tag=None)
bkw                  :: rop: ≈2^210.6, m: ≈2^196.4, mem: ≈2^197.4, b: 3, t1: 0, t2: 41, ℓ: 2, #cod: 943, #top: 0, #test: 82, tag: coded-bkw
usvp                 :: rop: ≈2^135.4, red: ≈2^135.4, δ: 1.004194, β: 371, d: 1869, tag: usvp
bdd                  :: rop: ≈2^155.7, red: ≈2^124.9, svp: ≈2^155.7, β: 333, η: 482, d: 1831, tag: bdd
dual                 :: rop: ≈2^138.7, mem: ≈2^89.1, m: 936, β: 379, d: 1960, ↻: 1, tag: dual
dual_hybrid          :: rop: ≈2^129.9, red: ≈2^129.9, guess: ≈2^123.2, β: 347, p: 2, ζ: 10, t: 90, β': 358, N: ≈2^74.0, m: 1024
┌─────────────────────────╥───────────────────────────────┐
│ n                       ║ 1024                          |
├─────────────────────────╫───────────────────────────────┤
│ q                       ║ 18446744073709551616          |
├─────────────────────────╫───────────────────────────────┤
│ secret key distributio

In [20]:
rlwe_param = LWE.Parameters(n=N, q=Q, Xs=rlwe_key_distributionn, Xe=e_rlwe_distribution)
print(rlwe_param)
try:
    # rlwe_result = LWE.estimate.rough(
    #     rlwe_param,
    #     jobs=jobs,
    # )
    # "arora-gb","bkw","usvp","bdd","bdd_hybrid","bdd_mitm_hybrid","dual","dual_hybrid","dual_mitm_hybrid"
    rlwe_result = LWE.estimate(
        rlwe_param,
        # red_shape_model=Simulator.CN11,
        deny_list=(
            # "arora-gb",
            # "bkw",
        ),
        jobs=jobs,
    )
except Exception as err:
    print("err=", err)
    print("Error Occur!")
else:
    rlwe_security = min([math.log(res.get("rop", 0), 2) for res in rlwe_result.values()])
    header = ["N","Q","logQ","secret key distribution","noise distribution", "rlwe security"]
    data = [N,Q,logQ,rlwe_key_distributionn,e_rlwe_distribution,rlwe_security]
    print(table(columns = [data], header_column = header, frame = True))

LWEParameters(n=2048, q=4611686018427322369, Xs=D(σ=0.50, μ=0.50), Xe=D(σ=3276.80), m=+Infinity, tag=None)
bkw                  :: rop: ≈2^331.3, m: ≈2^315.2, mem: ≈2^316.2, b: 5, t1: 0, t2: 75, ℓ: 4, #cod: 1884, #top: 0, #test: 165, tag: coded-bkw
usvp                 :: rop: ≈2^130.6, red: ≈2^130.6, δ: 1.004373, β: 349, d: 3854, tag: usvp
bdd                  :: rop: ≈2^131.0, red: ≈2^131.0, svp: ≈2^124.3, β: 351, η: 369, d: 3635, tag: bdd
dual                 :: rop: ≈2^132.5, mem: ≈2^84.7, m: 1970, β: 352, d: 4018, ↻: 1, tag: dual
dual_hybrid          :: rop: ≈2^128.1, red: ≈2^128.0, guess: ≈2^123.2, β: 336, p: 2, ζ: 10, t: 90, β': 352, N: ≈2^71.6, m: ≈2^11.0
┌─────────────────────────╥───────────────────────────┐
│ N                       ║ 2048                      |
├─────────────────────────╫───────────────────────────┤
│ Q                       ║ 4611686018427322369       |
├─────────────────────────╫───────────────────────────┤
│ logQ                    ║ 62                  

In [30]:
key_switching = LWE.Parameters(n=n, q=Q, Xs=lwe_key_distributionn, Xe=key_switching_key_distribution)
print(key_switching)
try:
    # key_switching_result = LWE.estimate.rough(
    #     key_switching,
    #     jobs=jobs,
    # )
    # "arora-gb","bkw","usvp","bdd","bdd_hybrid","bdd_mitm_hybrid","dual","dual_hybrid","dual_mitm_hybrid"
    key_switching_result = LWE.estimate(
        key_switching,
        # red_shape_model=Simulator.CN11,
        deny_list=(
            # "arora-gb",
            # "bkw",
        ),
        jobs=jobs,
    )
except Exception as err:
    print("err=", err)
    print("Error Occur!")
else:
    key_switching_security = min([math.log(res.get("rop", 0), 2) for res in key_switching_result.values()])
    header = ["n","logQ","secret key distribution","noise distribution", "key switching key security"]
    data = [n,logQ,lwe_key_distributionn,key_switching_key_distribution,key_switching_security]
    print(table(columns = [data], header_column = header, frame = True))

LWEParameters(n=1024, q=4611686018427322369, Xs=D(σ=0.50, μ=0.50), Xe=D(σ=439804651110.40), m=+Infinity, tag=None)
bkw                  :: rop: ≈2^204.4, m: ≈2^190.3, mem: ≈2^191.3, b: 3, t1: 0, t2: 39, ℓ: 2, #cod: 934, #top: 0, #test: 90, tag: coded-bkw
usvp                 :: rop: ≈2^135.4, red: ≈2^135.4, δ: 1.004194, β: 371, d: 1869, tag: usvp
bdd                  :: rop: ≈2^156.0, red: ≈2^124.9, svp: ≈2^156.0, β: 333, η: 483, d: 1813, tag: bdd
dual                 :: rop: ≈2^138.7, mem: ≈2^89.1, m: 936, β: 379, d: 1960, ↻: 1, tag: dual
dual_hybrid          :: rop: ≈2^129.9, red: ≈2^129.9, guess: ≈2^123.2, β: 347, p: 2, ζ: 10, t: 90, β': 358, N: ≈2^74.0, m: 1024
┌────────────────────────────╥──────────────────────────────┐
│ n                          ║ 1024                         |
├────────────────────────────╫──────────────────────────────┤
│ logQ                       ║ 62                           |
├────────────────────────────╫──────────────────────────────┤
│ secret key dis